##Covid SQL Explortation
- Script was done using DataBricks

In [ ]:

#File location

file_location = "**filepath**"
file_type = "csv"

# CSV options

infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location).createOrReplaceTempView("covidVaccinations")

In [ ]:

#File location

file_location = "**filepath**"
file_type = "csv"

# CSV options

infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location).createOrReplaceTempView("covidDeaths")

In [ ]:
%sql

select *
from covidVaccinations
order by 3,4


In [ ]:
%sql

select *
from covidDeaths
order by 3, 4

In [ ]:
%sql 

-- Select data that we are going to be using

select 
    location
   ,date
   ,total_cases
   ,new_cases
   ,total_deaths
   ,population
from covidDeaths
 where continent is not null
 order by 1,2

In [ ]:
%sql 

-- Look at total cases vs total deaths

select 
    location
   ,date
   ,total_cases
   ,total_deaths
   ,(total_deaths/total_cases)*100 as death_percentage
from covidDeaths
 where continent is not null
 order by 1,2

In [ ]:
%sql 

/**
Look at total cases vs population.
Shows what percentage of population got covid
**/

select 
    location
    ,date
    ,total_cases
    ,population
    ,(total_cases/population)*100 as infection_rate
from covidDeaths
 where continent is not null
 order by 1,2

In [ ]:
%sql

-- looking at total cases vs population

select 
    location
    ,population
    ,MAX(total_cases) as highestinfection_count
    ,MAX((total_cases/population))*100 as populationinfected_percentage
from covidDeaths
 group by 
   location
   ,population
 order by 4 desc

In [ ]:
%sql

-- looking at countries with the highest death count per population

select 
  location
  ,MAX(cast(total_deaths as int)) as death_count
from covidDeaths
 where continent is not null
 group by location
 order by 2 desc

In [ ]:
%sql

-- breaking data down by continent

select 
    continent
   ,MAX(cast(total_deaths as int)) as TotalDeathCount
from covidDeaths
 where continent is not null
 group by continent
 order by 2 desc

In [ ]:
%sql 

-- Look at total cases vs total deaths globally

select 
    date
   ,sum(cast(new_deaths as int)) as new_deaths
   ,sum(cast(new_cases as int)) as new_cases
   ,(sum(cast(new_deaths as int))/sum(cast(new_cases as int)))*100 as death_percentage
from covidDeaths
 where continent is not null
 group by date
 order by 1

In [ ]:
%sql

select 
  dea.continent
 ,dea.location
 ,dea.date
 ,dea.population
 ,vac.new_vaccinations
 ,sum(cast (vac.new_vaccinations as int)) over (partition by dea.location order by dea.location, dea.date) as peoplevaccinated_rolling
from covidDeaths dea
 join covidVaccinations vac 
   on (dea.date = vac.date and dea.location = vac.location)
 where dea.continent is not null --and dea.location = 'Albania'
 order by 2,3

In [ ]:
%sql

-- Use CTE 

with 
  PopvsVac   
     (
      continent
     ,location
     ,date
     ,population
     ,new_vaccinations
     ,peoplevaccinated_rolling
     )
as
(
select 
    dea.continent
   ,dea.location
   ,dea.date
   ,dea.population
   ,vac.new_vaccinations
   ,sum(cast (vac.new_vaccinations as int)) over (partition by dea.location order by dea.location, dea.date) as peoplevaccinated_rolling
from covidDeaths dea
 join covidVaccinations vac 
   on (dea.date = vac.date and dea.location = vac.location)
 where dea.continent is not null
)

select 
  *
 ,(RollingPeopleVaccinated/Population)*100 as populationvaccinated_percentage
from PopvsVac

In [ ]:
%sql

-- Temp view to store data 

create or replace temp view PercentPopulationVaccinated as 

select 
    dea.continent
   ,dea.location
   ,dea.date
   ,dea.population
   ,vac.new_vaccinations
   ,sum(cast (vac.new_vaccinations as int)) over (partition by dea.location order by dea.location, dea.date) as peoplevaccinated_rolling
from covidDeaths dea
 join covidVaccinations vac on (dea.date = vac.date and dea.location = vac.location)
 where dea.continent is not null
